# 패키지 임포트

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import nltk
import re
import string
import os
import wordninja
import json
import torch
from itertools import groupby
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from data_preprocessing import *
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# 데이터 로드

In [2]:
df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/textdataset/WELFake/WELFake_Dataset.csv')

# 전처리

In [3]:
df = df.dropna()
df.isnull().sum()

df.drop(columns=['Unnamed: 0'],inplace=True)

df.drop_duplicates(inplace=True)

X = df.drop(columns=['label'])
y = df['label']

X[['title','text']] = X[['title','text']].applymap(lambda x:remove_punctuation(x))
X[['title','text']] = X[['title','text']].applymap(lambda x:x.lower())

X=X['title']+' '+X['text']

X = X.apply(clean_text)
X = list(X)

pattern = '[^a-z ]'
Clean_X=[]
for sen in X:
    Clean_X.append(re.sub(pattern, '', str(sen)))

clean_df = pd.DataFrame({'Clean_X': Clean_X, 'y': y})

fake_df = clean_df[clean_df['y'] == 0]
real_df = clean_df[clean_df['y'] == 1]

fake_x=list(fake_df['Clean_X'])
real_x=list(real_df['Clean_X'])

real_selected_lst = []
fake_selected_lst = []

for sen in real_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        real_selected_lst.append(sen)
        
for sen in fake_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        fake_selected_lst.append(sen)
        
X=real_selected_lst[:10000]+fake_selected_lst[:10000]
y=[0]*10000+[1]*10000

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
to_txt=x_train+x_test

In [4]:
to_txt_split=[]
for i in to_txt:
    to_txt_split.append(split_compound_words(i))
    
to_txt=to_txt_split

In [5]:
to_txt=limit_words(to_txt,300) #버트 임베딩은 최대 임베딩 토큰 수가 정해져 있으므로 최대길이를 300으로 제한

In [6]:
to_txt

['lol cnn interview abruptly ends when student who started protest at betsy devo s speech realizes he cant explain why he s protesting video the students of the historically black be thune cook man university booed the invited commencement speaker education secretary betsy devo s they yelled over her speech they even turned their backs on her for minutes the students at be thune cook man reminded us just how far obama has divided our nation a sophomore student at be thune cook man a historically black university filmed himself running around with his phone as he yelled for the seniors to stand and rebuke betsy devo s during her commencement speech he implored the graduating seniors to disrupt her speech by yelling and standing with their backs to devo s who bravely persevered through her speech in spite of their embarrassing behavior the student who says his name is bobbie luke and identifies himself as a stupid nigga on his twitter account was eventually thrown out of the venue by cam

# 사전 학습 BERT 가져오기

In [7]:
def split_continuous_integers(lst):                                      #연속된 정수리스트를 split 해주는 함수
    for k, g in groupby(enumerate(lst), lambda i_x:i_x[0]-i_x[1]):
        yield list(map(itemgetter(1), g))
        
def add_previous_number(lst):                                            #최초 서브토큰 인덱스 추가
    return [[sub_lst[0] - 1] + sub_lst for sub_lst in lst]

def bert_word_embedding(text):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화
    indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    # Python list를 PyTorch tensor로 변환하기 
    tokens_tensor = torch.tensor([indexed_text])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors )
    token_embeddings = torch.stack(encoded_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
    token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
    token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])
    
    token_vecs_sum = []
    for token in token_embeddings:                #인코더의 마지막 4개의 은닉 상태를 합쳐 최종 벡터(4개 합친 것이 성능이 가장 좋음)
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(np.array(sum_vec))
    
    token_vecs_sum=np.array(token_vecs_sum)
    #서브토큰을 결합해 단어에 대한 임베딩 벡터를 얻자
    subword_indices = [i for i, token in enumerate(tokenized_text) if '##' in token]
    index_list = add_previous_number(list(split_continuous_integers(subword_indices))) #index_list는 서브토큰에 해당하는 인덱스를 한 리스트에 묶어줌
    
    new_token_vecs_sum = []
    last_index = 0

    for subword_inx_list in index_list:
        # 이전 인덱스와 현재 인덱스 그룹 사이의 벡터를 추가합니다.
        new_token_vecs_sum.extend(token_vecs_sum[last_index:subword_inx_list[0]])

        # 현재 인덱스 그룹에 해당하는 벡터의 평균을 계산하고 추가합니다.
        avg_vecs = np.mean(token_vecs_sum[subword_inx_list], axis=0)
        new_token_vecs_sum.append(avg_vecs)

        # 마지막 인덱스를 업데이트합니다.
        last_index = subword_inx_list[-1] + 1

    # 마지막 인덱스 이후의 벡터를 추가합니다.
    new_token_vecs_sum.extend(token_vecs_sum[last_index:])

    # 결과를 numpy array로 변환합니다.
    new_token_vecs_sum = np.array(new_token_vecs_sum[1:-1])
    
    return new_token_vecs_sum

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 미리 학습된 모델(가중치) 불러오기
model = BertModel.from_pretrained('bert-base-uncased')

text = to_txt[5]

marked_text = "[CLS] " + text + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화

indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)

segments_ids = [1] * len(tokenized_text)

# Python list를 PyTorch tensor로 변환하기 
tokens_tensor = torch.tensor([indexed_text])
segments_tensors = torch.tensor([segments_ids])

with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors )
    
token_embeddings = torch.stack(encoded_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])

token_vecs_sum = []

for token in token_embeddings:
    
    sum_vec = torch.sum(token[-4:], dim=0)
    
    token_vecs_sum.append(np.array(sum_vec))
    

token_vecs_sum=np.array(token_vecs_sum)

token_vecs_sum=bert_word_embedding(text)
token_vecs_sum.shape

(300, 768)

임베딩

In [9]:
x_train=to_txt[:16000]
x_test=to_txt[16000:]

In [ ]:
except_lst=[]
for i in range(len(x_train)):
    try:
        text = to_txt[i]
        embed_matrix=bert_word_embedding(text)       
        np.save('./bert_embedding/train(bert)/doc%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(300, 768)
(300, 768)
(300, 768)
(66, 768)
(300, 768)
(300, 768)
(300, 768)
(55, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(27, 768)
(300, 768)
(114, 768)
(300, 768)
(300, 768)
(300, 768)
(184, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(121, 768)


(228, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(73, 768)
(300, 768)
(101, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(124, 768)
(300, 768)
(300, 768)
(154, 768)
(190, 768)
(300, 768)
(300, 768)
(224, 768)
(204, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(52, 768)
(300, 768)
(285, 768)
(87, 768)
(300, 768)
(300, 768)
(255, 768)
(300, 768)
(54, 768)
(134, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(157, 768)
(81, 768)
(123, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(266, 768)
(300, 768)
(300, 768)
(269, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(66, 768)
(259, 768)
(300, 768)
(87, 768)
(300, 768)
(101, 768)
(59, 768)
(107, 7

(140, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(184, 768)
(63, 768)
(300, 768)
(252, 768)
(174, 768)
(300, 768)
(300, 768)
(114, 768)
(92, 768)
(300, 768)
(127, 768)
(81, 768)
(300, 768)
(154, 768)
(112, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(294, 768)
(300, 768)
(187, 768)
(189, 768)
(248, 768)
(300, 768)
(300, 768)
(295, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(161, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(92, 768)
(285, 768)
(107, 768)
(124, 768)
(210, 768)
(175, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(148, 768)
(300, 768)
(300, 768)
(300, 768)
(111, 768)
(280, 768)
(300, 768)
(300, 768)
(179, 768)
(237, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(259, 768)
(300, 768)
(300, 768)
(11

(300, 768)
(290, 768)
(53, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(200, 768)
(216, 768)
(300, 768)
(279, 768)
(55, 768)
(300, 768)
(300, 768)
(218, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(108, 768)
(300, 768)
(200, 768)
(300, 768)
(300, 768)
(226, 768)
(300, 768)
(300, 768)
(255, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(180, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(240, 768)
(36, 768)
(300, 768)
(300, 768)
(42, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(256, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(228, 768)
(47, 768)
(300, 768)
(300, 768)
(194, 768)
(228, 768)
(300, 768)
(300, 768)
(265, 768)
(300, 768)
(139, 768)
(300, 768)
(270, 768)
(165, 768)
(189, 768)
(300, 768)
(27, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(267, 768)
(300, 768)
(300, 768)
(300, 768)
(95, 768)
(300, 768)
(300, 

(300, 768)
(300, 768)
(300, 768)
(21, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(226, 768)
(300, 768)
(162, 768)
(300, 768)
(150, 768)
(250, 768)
(300, 768)
(300, 768)
(223, 768)
(66, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(298, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(123, 768)
(300, 768)
(300, 768)
(300, 768)
(85, 768)
(148, 768)
(258, 768)
(278, 768)
(300, 768)
(300, 768)
(252, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(117, 768)
(81, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(204, 768)
(300, 768)
(300, 768)
(300, 768)
(286, 768)
(300, 768)
(173, 768)
(300, 768)
(41, 768)
(300, 768)
(300, 768)
(116, 768)
(265, 768)
(300, 768)
(26, 768)
(298, 768)
(263, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(241, 768)
(300, 768)
(121, 768)
(300, 768)
(300, 768)
(90, 768)
(300, 768)
(102, 768)
(300, 768)
(300, 768)
(300, 768)
(52, 7

...

(300, 768)
(280, 768)
(300, 768)
(300, 768)
(36, 768)
(300, 768)
(300, 768)
(300, 768)
(247, 768)
(233, 768)
(258, 768)
(300, 768)
(300, 768)
(158, 768)
(300, 768)
(300, 768)
(117, 768)
(300, 768)
(300, 768)
(300, 768)
(99, 768)
(300, 768)
(300, 768)
(225, 768)
(285, 768)
(226, 768)
(238, 768)
(300, 768)
(238, 768)
(173, 768)
(300, 768)
(300, 768)
(258, 768)
(300, 768)
(300, 768)
(300, 768)
(23, 768)
(300, 768)
(300, 768)
(179, 768)
(300, 768)
(300, 768)
(114, 768)
(300, 768)
(300, 768)
(266, 768)
(300, 768)
(94, 768)
(244, 768)
(300, 768)
(300, 768)
(268, 768)
(300, 768)
(229, 768)
(300, 768)
(295, 768)
(262, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(37, 768)
(95, 768)
(217, 768)
(300, 768)
(300, 768)
(300, 768)
(198, 768)
(300, 768)
(143, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(41, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(149, 768)
(223, 768)
(300, 768)
(300, 768)
(252, 

(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(276, 768)
(300, 768)
(300, 768)
(119, 768)
(300, 768)
(300, 768)
(300, 768)
(133, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(286, 768)
(300, 768)
(300, 768)
(271, 768)
(300, 768)
(300, 768)
(258, 768)
(300, 768)
(137, 768)
(46, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(49, 768)
(300, 768)
(300, 768)
(222, 768)
(300, 768)
(300, 768)
(288, 768)
(300, 768)
(300, 768)
(146, 768)
(300, 768)
(300, 768)
(67, 768)
(300, 768)
(300, 768)
(300, 768)
(45, 768)
(300, 768)
(281, 768)
(300, 768)
(289, 768)
(300, 768)
(300, 768)
(58, 768)
(292, 768)
(287, 768)
(107, 768)
(186, 768)
(300, 768)
(30, 768)
(279, 768)
(83, 768)
(300, 768)
(300, 768)
(300, 768)
(261, 768)
(69, 768)
(84, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(94, 768)
(300, 768)
(300, 768)
(115, 768)
(258, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768

(300, 768)
(120, 768)
(300, 768)
(300, 768)
(184, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(109, 768)
(300, 768)
(145, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(208, 768)
(300, 768)
(300, 768)
(236, 768)
(300, 768)
(300, 768)
(122, 768)
(152, 768)
(40, 768)
(46, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(203, 768)
(300, 768)
(300, 768)
(300, 768)
(213, 768)
(300, 768)
(300, 768)
(93, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(261, 768)
(300, 768)
(59, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(264, 768)
(300, 768)
(300, 768)
(244, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(251, 768)
(125, 768)
(30

(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(220, 768)
(76, 768)
(209, 768)
(300, 768)
(85, 768)
(300, 768)
(300, 768)
(300, 768)
(280, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(81, 768)
(300, 768)
(300, 768)
(300, 768)
(263, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(206, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(258, 768)
(300, 768)
(300, 768)
(300, 768)
(32, 768)
(300, 768)
(87, 768)
(300, 768)
(162, 768)
(300, 768)
(300, 768)
(300, 768)
(79, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(80, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(221, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(265, 768)
(300, 768)
(300, 768)
(300, 

(300, 768)
(300, 768)
(300, 768)
(300, 768)
(236, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(224, 768)
(300, 768)
(300, 768)
(300, 768)
(23, 768)
(300, 768)
(187, 768)
(228, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(210, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(231, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(69, 768)
(300, 768)
(300, 768)
(82, 768)
(298, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(192, 768)
(300, 768)
(300, 768)
(44, 768)
(300, 768)
(122, 768)
(300, 768)
(300, 768)
(263, 768)
(300, 768)
(300, 768)
(94, 768)
(300, 768)
(84, 768)
(300, 768)
(300, 768)
(300, 768)
(87, 768)
(300, 768)
(300, 768)
(38, 768)
(227, 768)
(300, 768)
(45, 768)
(300, 768)
(300, 768)
(213, 768)
(165, 768)
(201, 768)
(300, 768)
(46, 768)
(101, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(90, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)

In [14]:
import pickle

print('제외된 문서의 수: ',len(except_lst))

with open('except_lst.pkl', 'wb') as file:
    pickle.dump(except_lst, file)

print("except_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  0
except_lst를 피클 파일로 저장했습니다.


In [15]:
# new_except_lst=[]
# to_txt_long_train=[]
# for idx in except_lst:
#     to_txt_long_train.append(to_txt[idx])
    
# to_txt_long_train=limit_words(to_txt_long_train,200)

# for i in except_lst:
#     try:
#         cnt=0
#         text = to_txt_long_train[cnt]
#         cnt+=1
#         embed_matrix=bert_word_embedding(text)   
#         np.save('./bert_embedding/train(bert)/doc%d'%i,embed_matrix)
#         print(embed_matrix.shape)
#     except:
#         new_except_lst.append(i)   #단어를 -개로 제한했는데도 sub_token 길이가 512가 넘는 문서는 new_except_lst에 추가

(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)


In [16]:
# new_except_lst

[]

In [15]:
except_test_lst=[]

for i in range(len(x_test)):
    try:
        text = to_txt[i+16000]
        embed_matrix=bert_word_embedding(text)       
        np.save('./bert_embedding/test(bert)/test%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_test_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(64, 768)
(300, 768)
(250, 768)
(300, 768)
(300, 768)
(229, 768)
(278, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(216, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(143, 768)
(300, 768)
(300, 768)
(173, 768)
(300, 768)
(300, 768)
(131, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(231, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(258, 768)
(300, 768)
(300, 768)
(14, 768)
(300, 768)
(300, 768)
(158, 768)
(300, 768)
(255, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(59, 768)
(300, 768)
(300, 768)
(239, 768)
(300, 768)
(300, 768)
(300, 768)
(237, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(122, 768)
(300, 768)
(300, 768)
(300, 768)
(82, 768)
(300, 768)
(195, 768)
(300, 768)
(164, 768)
(300, 768)
(300, 768)
(300, 768)
(278, 768)
(30

(300, 768)
(282, 768)
(300, 768)
(300, 768)
(238, 768)
(76, 768)
(297, 768)
(154, 768)
(70, 768)
(300, 768)
(300, 768)
(257, 768)
(300, 768)
(146, 768)
(300, 768)
(186, 768)
(300, 768)
(52, 768)
(69, 768)
(300, 768)
(300, 768)
(300, 768)
(265, 768)
(300, 768)
(300, 768)
(300, 768)
(115, 768)
(300, 768)
(300, 768)
(300, 768)
(209, 768)
(277, 768)
(300, 768)
(300, 768)
(78, 768)
(242, 768)
(256, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(13, 768)
(300, 768)
(300, 768)
(300, 768)
(69, 768)
(300, 768)
(300, 768)
(300, 768)
(132, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(129, 768)
(279, 768)
(300, 768)
(300, 768)
(101, 768)
(300, 768)
(140, 768)
(219, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(268, 768)
(278, 768)
(300, 768)
(300, 768)
(13, 768)
(300, 768)
(177, 768)
(111, 768)
(126, 768)
(300, 768)
(300, 768)
(141, 768)
(275, 768)
(226, 768)
(300, 7

(300, 768)
(137, 768)
(300, 768)
(300, 768)
(128, 768)
(300, 768)
(155, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(207, 768)
(276, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(28, 768)
(114, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(78, 768)
(54, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(39, 768)
(76, 768)
(300, 768)
(300, 768)
(136, 768)
(300, 768)
(300, 768)
(300, 768)
(216, 768)
(300, 768)
(221, 768)
(300, 768)
(213, 768)
(300, 768)
(91, 768)
(254, 768)
(298, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(83, 768)
(300, 768)
(250, 768)
(300, 768)
(296, 768)
(114, 768)
(300, 768)
(300, 768)
(155, 768)
(300, 768)
(224, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(292, 768)
(255, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(101, 768)
(300, 

(300, 768)
(300, 768)
(183, 768)
(300, 768)
(109, 768)
(300, 768)
(81, 768)
(300, 768)
(116, 768)
(300, 768)
(300, 768)
(282, 768)
(128, 768)
(300, 768)
(300, 768)
(157, 768)
(274, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(277, 768)
(300, 768)
(300, 768)
(300, 768)
(204, 768)
(300, 768)
(300, 768)
(243, 768)
(300, 768)
(300, 768)
(94, 768)
(300, 768)
(300, 768)
(234, 768)
(162, 768)
(300, 768)
(288, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(189, 768)
(274, 768)
(300, 768)
(91, 768)
(80, 768)
(300, 768)
(145, 768)
(300, 768)
(300, 768)
(300, 768)
(106, 768)
(300, 768)
(300, 768)
(10, 768)
(300, 768)
(300, 768)
(159, 768)
(300, 768)
(218, 768)
(300, 768)
(92, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(148, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(192, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(198, 768)
(78, 768)
(300, 

(300, 768)
(295, 768)
(300, 768)
(62, 768)
(300, 768)
(300, 768)
(300, 768)
(248, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(143, 768)
(300, 768)
(22, 768)
(201, 768)
(269, 768)
(300, 768)
(300, 768)
(300, 768)
(294, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(84, 768)
(195, 768)
(300, 768)
(300, 768)
(123, 768)
(83, 768)
(300, 768)
(300, 768)
(300, 768)
(92, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(204, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(222, 768)
(300, 768)
(300, 768)
(247, 768)
(11, 768)
(26, 768)
(300, 768)
(300, 768)
(300, 768)
(243, 768)
(75, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(154, 768)
(300, 768)
(200, 768)
(221, 768)
(300, 768)
(300, 768)
(46, 768)
(300, 768)
(197, 768)
(300, 76

(97, 768)
(61, 768)
(300, 768)
(144, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(107, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(110, 768)
(300, 768)
(300, 768)
(300, 768)
(181, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(97, 768)
(300, 768)
(300, 768)
(300, 768)
(227, 768)
(60, 768)
(300, 768)
(171, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(89, 768)
(91, 768)
(300, 768)
(74, 768)
(285, 768)
(300, 768)
(82, 768)
(300, 768)
(300, 768)
(199, 768)
(300, 768)
(300, 768)
(300, 768)
(107, 768)
(156, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(31, 768)
(300, 768)
(106, 768)
(300, 768)
(279, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(17, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(50, 768)
(300, 768)
(300, 768)
(300, 768)
(293, 768)
(300, 768)
(32, 768)
(300, 768)
(176, 768)
(199, 768)
(257, 768)
(300, 768)
(300, 768)
(300, 768)
(102, 768)
(220, 768)
(300, 768)
(300, 768)


In [16]:
import pickle

print('제외된 문서의 수: ',len(except_test_lst))

with open('except_test_lst.pkl', 'wb') as file:
    pickle.dump(except_test_lst, file)

print("except_test_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  0
except_test_lst를 피클 파일로 저장했습니다.


In [23]:
# new_except_test_lst=[]
# to_txt_long_test=[]
# for idx in except_test_lst:
#     to_txt_long_test.append(to_txt[idx+16000])
    
# to_txt_long_test=limit_words(to_txt_long_test,180)

# for i in except_test_lst:
#     try:
#         cnt=0
#         text = to_txt_long_test[cnt]
#         cnt+=1
#         embed_matrix=bert_word_embedding(text)       
#         np.save('./bert_embedding/test(bert)/test%d'%i,embed_matrix)
#         print(embed_matrix.shape)
#     except:
#         new_except_test_lst.append(i)   #단어를 -개로 제한했는데도 sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)


In [25]:
#new_except_test_lst

[]